In [118]:
# collapsed NNGP 

import numpy as np 
import pandas as pd
import scipy.sparse  as ss

In [209]:
# get indices of nearest neighbors for each vector (row)
# This is to calculate A, D from C
# A is strictly lower triangular matrix
# D is diagonal matrix


# X is n-by-p matrix
# n: number of spots
# p: number of elements for each spot
# X needs to be first ordered by the mean of the vector in ascending(descending) order

n = 20
p = 8
X = np.random.random_integers(20, size=(n, p))
X_sorted = np.array( sorted(X, key = lambda v: np.mean(v) ))
#X_sorted = sorted(X, key = lambda v: np.mean(v) )


# m neighbors of each row/spot
m = 5
nnindex = {}
for i in range(n):
    if i < m:
        nnindex[i] = np.array(range(i))
    else:
        sed = np.array([float('inf') for s in range(m)]) # initialize squared euclidian distance
        nn = np.array([0 for s in range(m)]) 
        j = 0   
        while j < i:
            distance = np.sum((X_sorted[i] - X_sorted[j]) ** 2)
            if distance < sed[m-1]:
                sed[m-1] = distance
                nn[m-1] = j
                sort_index = np.argsort(sed)#.astype(int)
                sed = sed[sort_index]
                nn = nn[sort_index]
            j += 1
        nnindex[i] = nn


# For the covariance matrix C
# C is n-by-n matrix
# A is 
# C = given ahead 
## Using pseudo code 2 from efficiency paper 
C = np.cov(X_sorted)
A = np.zeros((n,n)) # Initialize A 
D = ss.identity(n).toarray()


for i in range(1, n-1):
    A[i+1, nnindex[i+1]] = np.linalg.solve(C[nnindex[i+1]][:,  nnindex[i+1]], C[nnindex[i+1]][:, i+1] ) # A[i+1,N[i+1]] = solve(C[N[i+1],N[i+1]], C[N[i+1],i+1])
    D[i+1, i+1] = C[i+1, i+1] - np.inner(C[i+1][nnindex[i+1]], A[i+1][nnindex[i+1]]) # D[i+1, i+1] = C[i+1,i+1] - dot(C[i+1, N[i+1]], A[i+1,N[i+1]])


/var/folders/47/5_b8qynj1q5f_jwc87syz7_r0000gn/T/ipykernel_25479/1754466221.py:14: DeprecationWarning: This function is deprecated. Please call randint(1, 20 + 1) instead
  X = np.random.random_integers(20, size=(n, p))


In [215]:
np.random.seed(1234)
log_lambda_sorted = np.random.normal(loc = 10, scale = 5, size=n)  
log_lambda_sorted = log_lambda - min(log_lambda)  # log_lambda is given  
beta = np.random.standard_normal(size=p)  # beta is given

## algorithm 1 
# Update theta, tau, and epsilon 
# Liimt both tau and epsilon being positive


## Step 1
tau_a, tau_b, epsilon_a, epsilon_b, theta_a, theta_b = 2, 20, 2, 10, 2, 20

theta = tau = epsilon = 2 # initialize values 
epsilon_inverse = 1/epsilon 
tau_inverse = 1/tau
Omega = (np.identity(n) - A).T @ np.linalg.solve(D, np.identity(n) - A) + epsilon_inverse * np.identity(n)
L = np.linalg.cholesky(Omega)
r = log_lambda_sorted - X_sorted @ beta 
u = np.linalg.solve(L, r)
v = np.linalg.solve(L.T, u)
q = (np.dot(r, r) * epsilon_inverse - np.dot(r, v) * epsilon_inverse ** 2) * tau_inverse
d = (tau * epsilon) ** n * np.prod(np.diag(D)) * np.prod(np.diag(L)) ** 2
## Metrapolis-Hasting to sample new theta, tau, and epislon
## Todo: how to choose q(theta_cand | theta_(i-1)) for M-H algo 
##       rn use the random walk on the prior distribution
sample_size = 1 # predefined sample size --> only one sample ?
theta_sample = []
tau_sample = []
epsilon_sample = []
for s in range(sample_size):
    theta_cand = np.random.uniform(low=theta_a, high=theta_b, size = 1)
    #log_p_ratio = -q/2 + ----- # Todo: ??? need to recalculate the step 1 again for theta_cand  


## Step 2
# nu_beta, V_beta are the parameters in conjugate prior MVN for beta
nu_beta = np.array([1 for i in range(p)])
V_beta = np.identity(p) 
v2 = {}
for i in range(p):
    u2 = np.linalg.solve(L, X_sorted[:, i])
    v2[i] = np.linalg.solve(L.T, u2)
F = np.linalg.solve(V_beta, np.identity(p))
f = np.linalg.solve(V_beta, nu_beta)
b = np.zeros(p)
B = np.zeros((p, p))
for j in range(p):
    b[j] = (np.dot(log_lambda_sorted, X_sorted[:, j]) * epsilon_inverse - np.dot(log_lambda_sorted, v2[j]) * epsilon_inverse ** 2) * tau_inverse + f[j]
    for i in range(p):
        B[i,j] = ( np.dot(X_sorted[:, i], X_sorted[:, j]) * epsilon_inverse + np.dot(X_sorted[:, i], v2[j]) * epsilon_inverse ** 2 ) * tau_inverse + F[i,j]
beta = np.random.multivariate_normal(mean=np.linalg.solve(B, b), cov=np.linalg.solve(B, np.identity(p))) # update beta
